### RAG pipeline
- Indexing : 내부의 메모리(parametric) 뿐만 아니라, 외부의 메모리(non-parametric)를 모아서 사용하기 쉬운 상태로 가공하는 단계
- User input: 사용자가 질문하는 단계
- Retrieval: 이전에 가공해두었던 외부 메모리를 보며 user input과 유사한 지식을 골라내어 user input과 결합해 generator에 전달하는 단계
    - topK개의 유사한 chunk를 가져옴
    - TF-IDF: 문서에서 단어의 상대적 중요도를 평가하는데 사용
        - Term frequency: 해당 단어가 얼마나 등장하는지
        - Inverse document frequency: 전체 문서 집합에서 그 단어가 얼마나 더 나타나는지 측정. 
- Generator: 유저의 쿼리에 대한 답을 생성하는 단계

### KNN-LM (2019)
- 다음 단어 예측에 kNN모델 사용
- 드물게 등장하는 패턴을 다룰 때 유용
- decoder-only Transformer
    - 현재까지 입력된 내용을 바탕으로 가장 likely한 다음 토큰 예측
- 평가지표: perplexity(혼동정도)를 낮춤

### REALM (2020)
- Retrieval-Augmented Pre-training
- BERT Based
- REALM이 해결하고자하는 문제는 "QA(question answering)" 이다.
    - 질문이 주어졌을 때, 모델이 합당한 대답을 찾아내는 문제.
    - **open book QA: 모델이 정답이 포함된 문맥에 접근이 가능한 상태에서 답변을 생성하는 문제**
    - closed book QA: 모델이 외부의 지식없이 오직 parametric knowledge 만으로 답변을 생성하는 문제
    - open domain QA: 특정한 domain이 정해지지 않은 경우. (퀴즈쇼)
    - closed domain QA: 특정한 domain내의 질문인 경우. (학교 시험)
- REALM이 공략하는 것은 open book QA이다.
    - 공략법으로는 **Retrive-then-predict**를 사용. retriever-reader 구조 라고도 함.
    - 질문이 들어왔을 때, retriver가 해당 쿼리와 유사한 것을 탐색하여 reader에 전달.
    - reader는 전달받은 문서들로부터 정답을 추출.(생성X / 모델이 encoder only인 BERT임)
- REALM이 성능을 높이기 위해 시도한 것들
    - Salient span masking
        - 문장 내 랜덤한 부분이 아닌, 중요한 부분을 masking 처리
        - 이는 BERT-based Tagger 사용함.
    - Null document
        - 정답 추론에 문서가 필요하지 않을 때가 있음.
        - top-k doc에 null doc 포함.
    - Prohibiting trival retrievals
        - pre-training 기간 동안 trivial 후보 제외
    - Initialization
        - retriever: ICT trained BERT
        - encoder: uncased BERT-base        

### RAG (2020)
- Retrieval-Augmented Generator
- fine tuning only (pre-training X)
- BERT encoder, BART generator
    - BART: encoder-to-decoder 구조로 denoising 방식으로 학습된 생성형 모델임.
        - input이 들어오면 인코딩을 한번 거치고, 인코딩된 결과와 직전까지 생성된 토근을 통해 디코딩하는 모델임.  
- Retrieve-then-predict
    - reader는 전달받은 문서들로부터 정답을 생성. (추출x / 모델이 generator)
#### RAG-Token Model의 decoding 방법? 
- **beam search**
    - 매번 제일 확률이 큰 하나만 선택하고 나머지를 버리는 대신에, 매번 N개의 선택지를 고려함.
    - EOS를 만날 때까지 반복.
#### RAG-Squence Model의 decoding 방법?
- document가 두 개일 때,
    - 각 document(z1, z2)에 대해 sequence들이 생성됨. 이를 hypothesis라고 함.
        - 이들 중에는 여러 document에서 공통적으로 나타나는 sequence도 있고, 한 번만 나타나는 sequence도 있음.
    - 등장하지 않은 y,z상의 경우에 모두 확률이 0이라고 보는 Fast decoding 방식을 도입.